In [30]:
import pandas as pd
import numpy as np
from collections import defaultdict
from rapidfuzz import process, fuzz

In [31]:
file_path = r'Data\new data.xlsx'

excel_data = pd.ExcelFile(file_path)

dfs = {}

for sheet_name in excel_data.sheet_names:
    dfs[sheet_name] = pd.read_excel(excel_data, sheet_name=sheet_name, header=0)

for df_name, df in dfs.items():
    if 'Rk' in df.columns:
        df.drop(columns=['Rk'], inplace=True)

for sheet, df in dfs.items():
    print(f"Sheet name: {sheet}")
    print(df.head())

Sheet name: Linfield
               name category     team institution  Rk1  R1  \
0  Catherine Dudley     Open  CC SADU          CC  1st  77   
1       June LePage     Open  CC LEMC          CC  1st  80   
2  Spencer McDonald     Open  CC LEMC          CC  1st  79   
3      Charles Said     Open  CC SADU          CC  1st  77   
4    Brad Tomasovic     Open  CC THTO          CC  1st  77   

                side1                                         adj1  Rk2  R2  \
0  Opening Opposition    Frank StumboⒸ , Alex Cruz , Beth Anderson  1st  79   
1  Closing Government  Katie JonesⒸ , A Olszewski , Christopher Hu  2nd  77   
2  Closing Government  Katie JonesⒸ , A Olszewski , Christopher Hu  2nd  78   
3  Opening Opposition    Frank StumboⒸ , Alex Cruz , Beth Anderson  1st  78   
4  Opening Opposition        Brian SungⒸ , Ed Strok , Megan Towles  2nd  76   

   ...  R3               side3                                        adj3  \
0  ...  78  Closing Opposition  Kelly WelchⒸ , Christ

In [32]:
unique_names = set()

for df in dfs.values():
    if 'name' in df.columns:
        unique_names.update(df['name'].dropna().unique())

unique_names_list = list(unique_names)
master_df = pd.DataFrame(unique_names_list, columns=['name'])

name_mapping = {}
standard_names = []

for name in master_df['name']:
    if standard_names:
        match, score, _ = process.extractOne(name, standard_names, scorer=fuzz.WRatio)
        if score >= 85:
            name_mapping[name] = match
        else:
            standard_names.append(name)
            name_mapping[name] = name
    else:
        standard_names.append(name)
        name_mapping[name] = name

master_df['name'] = master_df['name'].replace(name_mapping)

for key, df in dfs.items():
    if 'name' in df.columns:
        df['name'] = df['name'].replace(name_mapping)

In [33]:
result_counts = defaultdict(lambda: defaultdict(int))

for key, df in dfs.items():
    df.columns = df.columns.str.strip()
    
    name_column = next((col for col in df.columns if 'name' in col.lower()), None)
    if not name_column:
        continue

    rank_columns = [col for col in df.columns if col.startswith('Rk')]
    
    for _, row in df.iterrows():
        name = row[name_column]
        for col in rank_columns:
            if row[col] in ['1st', '2nd', '3rd', '4th']:
                result_counts[name][row[col]] += 1

master_names = master_df['name'].tolist()
filtered_results = {name: result_counts[name] for name in master_names}

for rank in ['1st', '2nd', '3rd', '4th']:
    master_df[rank] = master_df['name'].map(lambda name: filtered_results.get(name, {}).get(rank, 0))

In [34]:
score_columns = ["R1", "R2", "R3", "R4", "R5"]

master_df['Total Accumulated Score'] = 0
master_df['Overall Average Score'] = 0.0
master_df['Count Appearances in DataFrames'] = 0

for col in score_columns:
    master_df[f'Total {col} Score'] = 0.0
    master_df[f'Count {col} Appearances'] = 0
    master_df[f'Average {col}'] = 0.0

for key, df in dfs.items():
    df.columns = df.columns.str.strip()
    
    name_column = next((col for col in df.columns if 'name' in col.lower()), None)
    if not name_column:
        continue

    available_columns = [col for col in score_columns if col in df.columns]
    for col in available_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    for _, row in df.iterrows():
        name = row[name_column]
        if name in master_df['name'].values:
            master_df.loc[master_df['name'] == name, 'Count Appearances in DataFrames'] += 1

            total_score = row[available_columns].sum()
            average_score = row[available_columns].mean()

            master_df.loc[master_df['name'] == name, 'Total Accumulated Score'] += total_score

            for col in available_columns:
                master_df.loc[master_df['name'] == name, f'Total {col} Score'] += row[col]
                master_df.loc[master_df['name'] == name, f'Count {col} Appearances'] += 1

for col in score_columns:
    master_df[f'Average {col}'] = (
        master_df[f'Total {col} Score'] / master_df[f'Count {col} Appearances']
    ).fillna(0).round(2)

master_df['Overall Average Score'] = (
    master_df['Total Accumulated Score'] / master_df['Count Appearances in DataFrames']
).fillna(0).round(2)

master_df.drop(
    [f'Total {col} Score' for col in score_columns]
    + [f'Count {col} Appearances' for col in score_columns]
    + ['Count Appearances in DataFrames'],
    axis=1,
    inplace=True
)

In [36]:
# Loop through each DataFrame in the dfs dictionary
for df_key, df in dfs.items():
    # Ensure the DataFrame contains a 'name' column before proceeding
    if 'name' in df.columns:
        # Iterate over each row in the DataFrame
        for _, row in df.iterrows():
            name = row['name']
            side = row['side1']  # Side from 'side1' column

            # If the name is found in the master_df, increment the corresponding side count
            if name in master_df['name'].values:
                # Update the count for the specific side
                master_df.loc[master_df['name'] == name, side] += 1

# After this, master_df will have counts for each side in their respective columns.